# Apyori / Apriori

In [1]:
import pandas as pd
import numpy as np
import os
import datetime
from apyori import apriori
import pandas as pd

os.chdir('/Users/arthur/Desktop/factor_investment')

In [2]:
market_data = [['T-Shirt','Pants','Jeans','Jersy','Socks','Basketball','Bottle','Shorts'],
 ['T-Shirt','Jeans'],
 ['Jersy','Basketball','Socks','Bottle'],
 ['Jeans','Pants','Bottle'],
 ['Shorts','Basketball'],
 ['Shorts','Jersy'],
 ['T-Shirt'],
 ['Basketball','Jersy']]
association_rules = apriori(market_data, min_support=0.2, min_confidence=0.2, min_lift=2, max_length=2)
association_results = list(association_rules)

for product in association_results:
    pair = product[0] 
    products = [x for x in pair]
    print(products)
    print("Rule: " + products[0] + " →" + products[1])
    print("Support: " + str(product[1]))
    print("Lift: " + str(product[2][0][3]))
    print("==================================")

['Basketball', 'Socks']
Rule: Basketball →Socks
Support: 0.25
Lift: 2.0
['Bottle', 'Pants']
Rule: Bottle →Pants
Support: 0.25
Lift: 2.6666666666666665
['Bottle', 'Socks']
Rule: Bottle →Socks
Support: 0.25
Lift: 2.6666666666666665
['Pants', 'Jeans']
Rule: Pants →Jeans
Support: 0.25
Lift: 2.6666666666666665
['Socks', 'Jersy']
Rule: Socks →Jersy
Support: 0.25
Lift: 2.0


In [2]:
col_list = ["證券代碼", "簡稱", "年月日", "收盤價(元)", "報酬率％"]
price_df = pd.read_csv('./data/TEJ_StockPrice_20052022.csv', encoding='big5', encoding_errors='ignore', usecols=col_list)
price_df["證券代碼"] = [str(x).strip() for x in price_df["證券代碼"]]
price_df["簡稱"] = [x.strip() for x in price_df["簡稱"]]

with open("./data/StockValidation.csv", 'r') as file:
    validstock_df = pd.read_csv(file, sep=',', low_memory=False, index_col=0)
    
validstock_df['stock_id'] = [str(x).strip() for x in validstock_df['stock_id']]

/var/folders/w8/jwxq4stj29506b1pf67xpb000000gq/T/ipykernel_6261/2000295121.py:2: DtypeWarning: Columns (0,9) have mixed types. Specify dtype option on import or set low_memory=False.
  price_df = pd.read_csv('./data/TEJ_StockPrice_20052022.csv', encoding='big5', encoding_errors='ignore', usecols=col_list)


In [3]:
price_df = price_df.loc[price_df.證券代碼.isin( validstock_df.iloc[:, 0] ), :]
price_df = price_df.reset_index(drop=True)
price_df['年月日'] = pd.to_datetime(price_df['年月日']).dt.date
price_df = price_df[ price_df['年月日'] > datetime.date(2009, 12,31) ]

In [4]:
price_df

,證券代碼,簡稱,年月日,收盤價(元),報酬率％
1242,50,元大台灣50,2010-01-04,38.1670,0.0886
1243,50,元大台灣50,2010-01-05,38.1670,0.0
1244,50,元大台灣50,2010-01-06,38.8763,1.8584
1245,50,元大台灣50,2010-01-07,38.7750,-0.2606
1246,50,元大台灣50,2010-01-08,39.0115,0.6098
...,...,...,...,...,...
3892485,9958,世紀鋼,2022-05-10,117.5000,1.2931
3892486,9958,世紀鋼,2022-05-11,115.5000,-1.7021
3892487,9958,世紀鋼,2022-05-12,112.0000,-3.0303
3892488,9958,世紀鋼,2022-05-13,116.0000,3.5714


In [5]:
Apr = pd.DataFrame(index = price_df['年月日'].unique())

for i in price_df['年月日'].unique():
#     print(i)
    daily_df = price_df [ (price_df['年月日'] == i) & (price_df['報酬率％'] > 0) ]
    Apr.loc[i, 0] = daily_df.證券代碼.values

In [6]:
# 檢查有沒有出錯
# for i in Apr.index:
#     print(i)
#     daily_df = price_df [ price_df['年月日'] == i ]
#     daily_df = daily_df[daily_df.證券代碼.isin(Apr.loc[i, 0])]
#     if sum(daily_df['報酬率％'] > 0) == len(daily_df['報酬率％']):
#         continue
#     else:
#         print(i)
#         break

In [7]:
Apr

,0
2010-01-04,"[50, 56, 1101, 1103, 1104, 1108, 1109, 1110, 1..."
2010-01-05,"[1101, 1102, 1109, 1110, 1201, 1210, 1215, 121..."
2010-01-06,"[50, 56, 1101, 1102, 1104, 1109, 1110, 1203, 1..."
2010-01-07,"[1108, 1109, 1218, 1225, 1233, 1303, 1310, 131..."
2010-01-08,"[50, 56, 1102, 1103, 1104, 1108, 1110, 1203, 1..."
...,...
2022-05-10,"[56, 1104, 1109, 1201, 1215, 1216, 1218, 1229,..."
2022-05-11,"[50, 1104, 1110, 1264, 1307, 1319, 1337, 1338,..."
2022-05-12,"[1259, 1477, 1522, 1535, 1538, 1605, 1808, 202..."
2022-05-13,"[50, 56, 1101, 1102, 1103, 1104, 1108, 1109, 1..."


In [8]:
type(Apr[0].values)

numpy.ndarray

In [9]:
association_rules = apriori(Apr[0].values, min_support=0.2, min_confidence=0.2, min_lift=2, max_length=2)
# association_rules = apriori(Apr[0].values, min_support=0.1, min_confidence=0.1, min_lift=2, max_length=5)
# association_results = list(association_rules)

In [10]:
# list(association_rules)

In [11]:
association_results = list(association_rules)

In [12]:
for product in association_results:
    pair = product[0] 
    products = [x for x in pair]
    print(products)
    print("Rule: " + products[0] + " <-> " + products[1])
    print("Support: " + str(product[1]))
    print("Lift: " + str(product[2][0][3]))
    print("==================================")

['3519', '3514']
Rule: 3519 <-> 3514
Support: 0.2211253701875617
Lift: 2.1156200145032633
['3561', '3514']
Rule: 3561 <-> 3514
Support: 0.24152681803224746
Lift: 2.393426897572909
['3519', '3561']
Rule: 3519 <-> 3561
Support: 0.22704837117472854
Lift: 2.238530641003059
['3519', '3579']
Rule: 3519 <-> 3579
Support: 0.2250740375123396
Lift: 2.11244340787488
